# 特殊メソッドgetstate, setstateの使い方




Pickle 化できないオブジェクトがメンバ変数に含まれる時点でお手上げかというとそうではない。 Python ではこの問題に対処するための方法が用意されている。 それが特殊メソッド __getstate__() と __setstate__() を使うやり方だ。 このふたつの特殊メソッドはオブジェクトが Pickle 化・非 Pickle 化される際の処理に対応している。 つまり、その挙動をオーバーライドできるということだ。

次のサンプルコードではユーザ定義クラス Sample に __getstate__() と __setstate__() を実装することで Pickle 化できるようにしている。 具体的な処理としては、Pickle 化されるときに呼ばれる __getstate__() で Pickle 化できないオブジェクトを Pickle 化の対象から除外している。 そして、非 Pickle 化されるときに呼ばれる __setstate__() では、ファイル名からファイルオブジェクトを改めて作りなおしている。 尚、非 Pickle 化される際には __init__() メソッドは呼ばれない点に注意が必要となる。

In [ ]:
import pickle


class Sample(object):

    def __init__(self, filename):
        """非 Pickle 化されるときは呼ばれない"""

        # 文字列は Pickle 化できる
        self.filename = filename

        # ファイルオブジェクトは Pickle 化できない
        self.file = open(filename, mode='rb')

    def __getstate__(self):
        """Pickle 化されるとき呼ばれる"""

        # オブジェクトの持つ属性をコピーする
        state = self.__dict__.copy()

        # Pickle 化できない属性を除去する
        del state['file']

        # Pickle 化する属性を返す
        return state

    def __setstate__(self, state):
        """非 Pickle 化されるとき呼ばれる"""

        # オブジェクトの持つ属性を復元する
        self.__dict__.update(state)

        # Pickle 化できなかった属性を作りなおす
        self.file = open(self.filename, mode='rb')


def main():
    obj = Sample('/dev/null')
    binary = pickle.dumps(obj)
    restored_obj = pickle.loads(binary)
    print(restored_obj.filename)
    print(restored_obj.file)


if __name__ == '__main__':
    main()